In [ ]:
from tensorflow.keras.models import load_model

# Load HDF5 model
model = load_model('soil_classification_model.h5')

In [ ]:
test_dir = '/kaggle/input/soil-classification/soil_classification-2025/test'

test_df = pd.read_csv('/kaggle/input/soil-classification/soil_classification-2025/test_ids.csv')
test_df['image_path'] = test_df['image_id'].apply(lambda x: f"{test_dir}/{x}")

test_df.head()

In [ ]:
from tensorflow.keras.preprocessing import image
import numpy as np

def load_and_preprocess(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = img_array / 255.0  # Normalize
    return img_array

# Preprocess all test images
X_test = np.array([load_and_preprocess(path) for path in test_df['image_path']])


In [ ]:
predictions = model.predict(X_test)

# If it's a classification model (e.g., softmax output), get class labels
predicted_labels = np.argmax(predictions, axis=1)

# Add predictions to the DataFrame
test_df['prediction'] = predicted_labels

In [ ]:
# class_indices is a dict like: {'clay': 0, 'red': 1, 'black': 2, 'alluvial': 3}
class_indices = train_data.class_indices

# Reverse it to map index to class
index_to_class = {v: k for k, v in class_indices.items()}

class_label ={
    0:'Alluvial soil',
    1:'Black Soil',
    2:'Clay soil',
    3:'Red soil'
}

In [ ]:
# Map encoded predictions to class labels
test_df['soil_type'] = test_df['prediction'].map(class_label)

# Save the correct columns to CSV
test_df[['image_id', 'soil_type']].to_csv('submission.csv', index=False)